In [18]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:

master_data_filepath = '..\\2024-12-04 Data summary v5 - Mar28 data, geographies.xlsx' 
transit_filepath = '..\\TransitStatsData_20250331.xlsx'
priority_pop_filepath = '..\\CHN_RequiredData_20250325.xlsx'

db_path = '..\\sources\\hicc.db'


In [21]:
# The following code uploads data to database
from DBUploader import DBUploader
uploader = DBUploader(master_data_filepath, transit_filepath, priority_pop_filepath, db_path)



Reading Input master data...
Preparing Province, CDs and CSDs data...
Preparing Output 1
Output 1 Successfully created...
Preparing Output 1
Output 1 Successfully created...
Preparing Output 2...
Output 2 Successfully created...
Preparing Output 3...
Output 3 Successfully created...
Preparing Output 4a...
Output 4a Successfully created...
Preparing Output 4b...
Output 4b Successfully created...
Preparing Output 5a...
Output 5a Successfully created...
Preparing Output 5b...
Output 5b Successfully created...
Preparing Output 6...
Output 6 Successfully created...
Preparing Output 7...
Output 7 Successfully created...
Preparing Output 8...
Output 8 Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Outpu

In [22]:
uploader()

Database ready....


### Test

In [3]:
from PrepareTables import PrepareTables
pt = PrepareTables(master_data_filepath, transit_filepath, priority_pop_filepath)


Reading Input master data...
Preparing Province, CDs and CSDs data...


In [5]:
pt.prepare_output_13()

Preparing Output 13...
> c:\users\himal\projects\24095-hart-hicc\helper\preparetables.py(689)prepare_output_13()
    687 
    688         pdb.set_trace()
--> 689         output_13['Net Change in Affordable Units Very Low'] = output_13["2016to2021_AffordableUnits_Built_VeryLowOnly"
    690                                                                          ] - output_13['2016to2021_AffordableUnits_Lost_VeryLowOnly']
    691         output_13['Net Change in Affordable Units Low'] = output_13["2016to2021_AffordableUnits_Built_LowOnly"

ipdb> c
Output 13 Successfully created...


,Geography,ALT_GEO_CODE_EN,2016to2021_AffordableUnits_Built,2016to2021_AffordableUnits_Lost,2016to2021_AffordableUnits_Built_VeryLowOnly,2016to2021_AffordableUnits_Lost_VeryLowOnly,2016to2021_AffordableUnits_Built_LowOnly,2016to2021_AffordableUnits_Lost_LowOnly,Net Change in Affordable Units Very Low,Net Change in Affordable Units Low,Net Change in Affordable Units
0,Arnold's Cove,1001270,0.0,-30.0,0.0,-20.0,0.0,-10.0,20.0,10.0,30.0
1,Avondale,1001464,0.0,20.0,0.0,0.0,0.0,20.0,0.0,-20.0,-20.0
2,Bauline,1001512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bay Bulls,1001557,0.0,-5.0,0.0,0.0,0.0,-5.0,0.0,5.0,5.0
4,Bay Roberts,1001409,30.0,-15.0,0.0,5.0,30.0,-20.0,-5.0,50.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...
302,Region 5 (6105) 01010 ( 12.8%),6105,20.0,165.0,10.0,95.0,10.0,70.0,-85.0,-60.0,-145.0
303,Region 6 (6106) 00000 ( 5.9%),6106,60.0,145.0,25.0,80.0,35.0,65.0,-55.0,-30.0,-85.0
304,Qikiqtaaluk (6204) 02020 ( 24.1%),6204,300.0,215.0,165.0,115.0,135.0,100.0,50.0,35.0,85.0
305,Kivalliq (6205) 01010 ( 16.9%),6205,125.0,60.0,75.0,110.0,50.0,-50.0,-35.0,100.0,65.0


In [ ]:
output_4a_long [output_4a_long ['Geography'] == 'Canada']

In [ ]:
pt.prepare_output_1()

In [24]:
# output_5a = p.prepare_output_5a()

# p.prepare_output_5b(output_5a)
# 2016to2021_AffordableUnits_Built
# 2016to2021_AffordableUnits_Lost 2016_AMHI
tb13 = pt.prepare_output_13()

Preparing Output 12...
Geography                            object
GEO_TYPE_ABBR_EN                     object
ALT_GEO_CODE_EN                       int64
PR_CODE_EN                            int64
2016to2021_AffordableUnits_Built    float64
2016to2021_AffordableUnits_Lost     float64
dtype: object
          Geography GEO_TYPE_ABBR_EN  ALT_GEO_CODE_EN  PR_CODE_EN  \
0     Arnold's Cove                T          1001270          10   
1          Avondale                T          1001464          10   
2           Bauline                T          1001512          10   
3         Bay Bulls                T          1001557          10   
4       Bay Roberts                T          1001409          10   
...             ...              ...              ...         ...   
3169       Kimmirut              HAM          6204005          62   
3170    Pangnirtung              HAM          6204009          62   
3171     Pond Inlet              HAM          6204020          62   
3172   Qi

In [75]:
tb13 = pt.prepare_output_13()


Preparing Output 13...
Output 13 Successfully created...


In [76]:
tb13

,Geography,GEO_TYPE_ABBR_EN,ALT_GEO_CODE_EN,PR_CODE_EN,2016to2021_AffordableUnits_Built,2016to2021_AffordableUnits_Lost,Net Change in Affordable Units
0,Arnold's Cove,T,1001270,10,0.0,-30.0,30.0
1,Avondale,T,1001464,10,0.0,20.0,-20.0
2,Bauline,T,1001512,10,0.0,0.0,0.0
3,Bay Bulls,T,1001557,10,0.0,-5.0,5.0
4,Bay Roberts,T,1001409,10,30.0,-15.0,45.0
...,...,...,...,...,...,...,...
3169,Kimmirut,HAM,6204005,62,0.0,-5.0,5.0
3170,Pangnirtung,HAM,6204009,62,0.0,5.0,-5.0
3171,Pond Inlet,HAM,6204020,62,40.0,-20.0,60.0
3172,Qikiqtarjuaq,HAM,6204010,62,0.0,-10.0,10.0


In [71]:
tb11 = pt.prepare_output_11()
# 'Youth', 'SameGender', 'TransgenderNonBinary', 'MentalHealth', 'Veteran'

Preparing Output 11...
Output 11 Successfully created...


In [72]:
tb11

,Geography,GEO_TYPE_ABBR_EN,ALT_GEO_CODE_EN,PR_CODE_EN,2021_CHN_Youth,2021_CHN_TransgenderNonBinary,2021_CHN_Veteran,2021_ExaminedForCHN_MentalHealth,2021_ExaminedForCHN_Veteran,2021_ExaminedForCHN_SameGender,2021_CHN_MentalHealth,2021_CHN_SameGender,2021_ExaminedForCHN_Youth,2021_ExaminedForCHN_TransgenderNonBinary,Youth_Rate of CHN,SameGender_Rate of CHN,TransgenderNonBinary_Rate of CHN,MentalHealth_Rate of CHN,Veteran_Rate of CHN
0,Arnold's Cove,T,1001270,10,0.0,NaN,0.0,30.0,0.0,NaN,0.0,NaN,15.0,NaN,0.000000,1.0,1.0,0.000000,NaN
1,Avondale,T,1001464,10,0.0,NaN,0.0,30.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0,1.0,0.000000,NaN
2,Bauline,T,1001512,10,0.0,NaN,0.0,30.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0,1.0,0.000000,NaN
3,Bay Bulls,T,1001557,10,0.0,NaN,0.0,85.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0,1.0,0.000000,NaN
4,Bay Roberts,T,1001409,10,25.0,NaN,0.0,240.0,0.0,NaN,25.0,NaN,115.0,NaN,0.217391,1.0,1.0,0.104167,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,Kimmirut,HAM,6204005,62,0.0,NaN,0.0,10.0,0.0,NaN,0.0,NaN,15.0,NaN,0.000000,1.0,1.0,0.000000,NaN
3170,Pangnirtung,HAM,6204009,62,20.0,NaN,0.0,25.0,0.0,NaN,15.0,NaN,35.0,NaN,0.571429,1.0,1.0,0.600000,NaN
3171,Pond Inlet,HAM,6204020,62,25.0,NaN,0.0,85.0,10.0,NaN,40.0,NaN,45.0,NaN,0.555556,1.0,1.0,0.470588,0.0
3172,Qikiqtarjuaq,HAM,6204010,62,15.0,NaN,0.0,10.0,0.0,NaN,0.0,NaN,20.0,NaN,0.750000,1.0,1.0,0.000000,NaN


In [21]:
tb12['2024_Coops'].describe()

count      295.000000
mean       307.976271
std       1231.013952
min          1.000000
25%         18.500000
50%         44.000000
75%        169.500000
max      17012.000000
Name: 2024_Coops, dtype: float64

In [3]:
def geo_code_extractor(geography):
    geo = geography.split()
    # print(geo)
    for g in geo:
        if g[0] == '(' and g[1].isdigit():
            g = g.replace("(", "")
            g = g.replace(")", "")
            break
    return g

In [4]:
geo_code_list = geo_code.apply(lambda x: geo_code_extractor(x))
region_code_list = geo_code_list.apply(lambda x: x[:4])
province_code_list = geo_code_list.apply(lambda x: x[:2])

In [8]:
geo_code_mapping = pd.DataFrame({'Geo_Code': geo_code_list, 'Region_Code': region_code_list, 'Province_Code': province_code_list, 'Geography': df_income['Formatted Name']})
geo_code_mapping['Geo_Code_Length'] = geo_code_mapping['Geo_Code'].apply(lambda x: len(x))

region_code_mapping = geo_code_mapping.loc[geo_code_mapping['Geo_Code_Length'] == 4, :]
province_code_mapping = geo_code_mapping.loc[geo_code_mapping['Geo_Code_Length'] <= 2, :]

mapped_geo_code = geo_code_mapping.merge(region_code_mapping[['Geo_Code','Geography']], how = 'left', left_on = 'Region_Code', right_on = 'Geo_Code')
mapped_geo_code = mapped_geo_code.merge(province_code_mapping[['Geo_Code','Geography']], how = 'left', left_on = 'Province_Code', right_on = 'Geo_Code')
mapped_geo_code = mapped_geo_code[['Geo_Code_x', 'Region_Code', 'Province_Code', 'Geography_x','Geography_y','Geography']]
mapped_geo_code.columns = ['Geo_Code', 'Region_Code', 'Province_Code', 'Geography','Region','Province']
# mapped_geo_code['Region'] = mapped_geo_code['Region'].fillna(mapped_geo_code['Province'])

In [23]:
mapped_geo_code[mapped_geo_code['Geography']=='Vancouver CY (CSD, BC)']

,Geo_Code,Region_Code,Province_Code,Geography,Region,Province
809,5915022,5915,59,"Vancouver CY (CSD, BC)","Greater Vancouver (CD, BC)",British Columbia (Province)


In [14]:
from sqlalchemy import create_engine, inspect
import os
import pandas as pd
# current_dir = os.path.dirname(os.path.abspath(__file__))
engine_new = create_engine('sqlite:///../sources//hicc.db')


In [15]:
output_1a = pd.read_sql_table('Output_1a', engine_new.connect())
output_1a
# inspector = inspect(engine_new)
# tables = inspector.get_table_names()
# tables

,pk


In [30]:
lis = [3,4,5,6, 'n/a']
if 'n/a' in lis:
    lis.remove('n/a')
lis

[3, 4, 5, 6]